In [ ]:
from pymongo import MongoClient
from pprint import pprint
import pandas as pd
client = MongoClient(port=27017)
db=client.insurance
cursor=db.quotes.aggregate([{"$group":{"_id":{"email":"$formData.email","category":"$category","product":"$product"}}},{"$project":{"email":"$_id.email","category":"$_id.category","product":"$_id.product","_id":0}}])

In [6]:
def suggest(category,product):
    client = MongoClient(port=27017)
    db=client.insurance
    cursor=db.quotes.aggregate([{"$group":{"_id":{"email":"$formData.email","category":"$category","product":"$product"}}},{"$project":{"email":"$_id.email","category":"$_id.category","product":"$_id.product","_id":0}}])
    data=pd.DataFrame(cursor)
    print(data)
    data["categoryProduct"]=data["category"]+","+data["product"]
    categoryProduct=set(data["categoryProduct"])
    outputData=list()
    for i in categoryProduct:
        iUsers=set(data.email[data["categoryProduct"]==i])    
        for j in categoryProduct:
            curOutput=list()
            curOutput.append(i)
            curOutput.append(j)
            jUsers=set(data.email[data["categoryProduct"]==j])
            curOutput.append(len(iUsers&jUsers))
            outputData.append(curOutput)
    outputData=pd.DataFrame(outputData,columns=["i","j","count"])
    categoryProduct=category+","+product
    processData=outputData[outputData["i"]==categoryProduct]
    #print(processData)
    print("length")
    print(len(set(data.email[data["categoryProduct"]==categoryProduct])))
    total=len(set(data.email[data["categoryProduct"]==categoryProduct]))
    if(total>0):
        processData["count"]=(processData["count"]*100)/total
        print(processData)
        categories=set(processData.j[processData["count"]>60])
        categoryList=list()
        productList=list()
        for cat in categories:
            split=cat.split(",")
            if(category!=split[0] and product!=split[1]):
                categoryList.append(split[0])
                productList.append(split[1])
        return list(db.formConfig.find({"category":{"$in":categoryList},"product":{"$in":productList}},{"category":1,"product":1,"image":1,"info":1,"_id":0}))
    return list()

In [7]:
suggest("Travel Insurance","Overseas Travel Insurance")

                            email             category  \
0      4sf15cs122@sahyadri.edu.in      Motor Insurance   
1             ramanonda@gmail.com      Motor Insurance   
2          prateek.jo97@gmail.com     Travel Insurance   
3       4sf15cs12@sahyadri.edu.in      Motor Insurance   
4      4sf15cs122@sahyadri.edu.in  Term Life Insurance   
5         mnrrnmproject@gmail.com  Term Life Insurance   
6       4sf15cs12@sahyadri.edu.in     Travel Insurance   
7       4sf15cs22@sahyadri.edu.in     Travel Insurance   
8       4sf15cs22@sahyadri.edu.in  Term Life Insurance   
9          prateek.jo97@gmail.com  Term Life Insurance   
10        mnrrnmproject@gmail.com      Motor Insurance   
11      4sf15cs12@sahyadri.edu.in  Term Life Insurance   
12          attri.param@gmail.com  Term Life Insurance   
13                            NaN     Travel Insurance   
14                            NaN     Health Insurance   
15          attri.param@gmail.com      Motor Insurance   
16       rashw

<ipython-input-6-10718179d280>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processData["count"]=(processData["count"]*100)/total


[{'image': 'https://images.moneycontrol.com/static-mcnews/2018/03/car-insurance-770x433.jpg',
  'product': 'Car Insurance',
  'category': 'Motor Insurance',
  'info': ''},
 {'image': 'https://brandsite-static.hdfclife.com/media/image/45.jpg',
  'product': 'Individual Life Insurance',
  'category': 'Term Life Insurance',
  'info': ''}]

In [5]:
suggest("Term Life Insurance","Individual Life Insurance")

length
10
                                               i  \
4  Term Life Insurance,Individual Life Insurance   
5  Term Life Insurance,Individual Life Insurance   
6  Term Life Insurance,Individual Life Insurance   
7  Term Life Insurance,Individual Life Insurance   

                                               j  count  
4       Health Insurance,Individual Health Guard    0.0  
5  Term Life Insurance,Individual Life Insurance  100.0  
6     Travel Insurance,Overseas Travel Insurance  100.0  
7                  Motor Insurance,Car Insurance  100.0  


<ipython-input-2-46248dbcc8a3>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processData["count"]=(processData["count"]*100)/total


[{'image': 'https://images.moneycontrol.com/static-mcnews/2018/03/car-insurance-770x433.jpg',
  'product': 'Car Insurance',
  'category': 'Motor Insurance',
  'info': ''},
 {'image': 'https://www.shriramgi.com/images/travelproductbg.jpg',
  'product': 'Overseas Travel Insurance',
  'category': 'Travel Insurance',
  'info': ''}]

In [ ]:
suggest("Health Insurance","Individual Health Guard" )